<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [58]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [59]:
# Posibles bandas
os.listdir("./songs_dataset/")

['adele.txt',
 'al-green.txt',
 'alicia-keys.txt',
 'amy-winehouse.txt',
 'beatles.txt',
 'bieber.txt',
 'bjork.txt',
 'blink-182.txt',
 'bob-dylan.txt',
 'bob-marley.txt',
 'britney-spears.txt',
 'bruce-springsteen.txt',
 'bruno-mars.txt',
 'cake.txt',
 'dickinson.txt',
 'disney.txt',
 'dj-khaled.txt',
 'dolly-parton.txt',
 'dr-seuss.txt',
 'drake.txt',
 'eminem.txt',
 'janisjoplin.txt',
 'jimi-hendrix.txt',
 'johnny-cash.txt',
 'joni-mitchell.txt',
 'kanye-west.txt',
 'kanye.txt',
 'Kanye_West.txt',
 'lady-gaga.txt',
 'leonard-cohen.txt',
 'lil-wayne.txt',
 'Lil_Wayne.txt',
 'lin-manuel-miranda.txt',
 'lorde.txt',
 'ludacris.txt',
 'michael-jackson.txt',
 'missy-elliott.txt',
 'nickelback.txt',
 'nicki-minaj.txt',
 'nirvana.txt',
 'notorious-big.txt',
 'notorious_big.txt',
 'nursery_rhymes.txt',
 'patti-smith.txt',
 'paul-simon.txt',
 'prince.txt',
 'r-kelly.txt',
 'radiohead.txt',
 'rihanna.txt']

In [60]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)
df.head()

C:\Users\Lorda\AppData\Local\Temp\ipykernel_29540\3849064916.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,"Yesterday, all my troubles seemed so far away"
1,Now it looks as though they're here to stay
2,"Oh, I believe in yesterday Suddenly, I'm not h..."
3,There's a shadow hanging over me.
4,"Oh, yesterday came suddenly Why she had to go ..."


In [61]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1846


### 1 - Preprocesamiento

In [62]:
from keras_preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [63]:
# Demos un vistazo
sentence_tokens[:2]

[['yesterday', 'all', 'my', 'troubles', 'seemed', 'so', 'far', 'away'],
 ['now', 'it', 'looks', 'as', 'though', "they're", 'here', 'to', 'stay']]

### 2 - Crear los vectores (word2vec)

In [64]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [65]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [66]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [67]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1846


In [68]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 445


### 3 - Entrenar embeddings

In [69]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 113208.15625
Loss after epoch 1: 65953.953125
Loss after epoch 2: 65934.265625
Loss after epoch 3: 65711.96875
Loss after epoch 4: 63872.5
Loss after epoch 5: 64142.46875
Loss after epoch 6: 64058.8125
Loss after epoch 7: 64755.4375
Loss after epoch 8: 62585.5625
Loss after epoch 9: 60415.25
Loss after epoch 10: 59823.625
Loss after epoch 11: 58930.1875
Loss after epoch 12: 57726.4375
Loss after epoch 13: 56493.125
Loss after epoch 14: 55842.5625
Loss after epoch 15: 55862.5
Loss after epoch 16: 51695.8125
Loss after epoch 17: 49815.5
Loss after epoch 18: 49568.125
Loss after epoch 19: 48959.75


(156986, 287740)

### 4 - Ensayar

In [70]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('pretty', 0.9087147116661072),
 ('sleep', 0.8649678230285645),
 ('help', 0.8594305515289307),
 ('try', 0.8456019759178162),
 ('cry', 0.8387227654457092),
 ('little', 0.8273633718490601),
 ('not', 0.8181102871894836),
 ('seems', 0.8175203204154968),
 ('twist', 0.8154979348182678),
 ('peace', 0.812263011932373)]

In [71]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('shake', -0.21786975860595703),
 ('our', -0.24495810270309448),
 ('come', -0.2613270878791809),
 ('on', -0.26722386479377747),
 ('bang', -0.27329546213150024),
 ('five', -0.2741836607456207),
 ('six', -0.2774038314819336),
 ('baby', -0.2776353061199188),
 ('work', -0.2790372967720032),
 ('four', -0.2792743146419525)]

In [72]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('five', 0.9786694049835205),
 ('three', 0.9761051535606384),
 ('six', 0.966891348361969),
 ('two', 0.9565720558166504),
 ('seven', 0.954232931137085),
 ('sixty', 0.9003373980522156),
 ('one', 0.816501259803772),
 ('us', 0.7762899398803711),
 ('crying', 0.772737979888916),
 ('strawberry', 0.7672452330589294)]

In [73]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[("can't", 0.9409592747688293),
 ('buy', 0.9399508237838745),
 ('much', 0.8996850848197937),
 ('hide', 0.8465201258659363),
 ('just', 0.8415144085884094)]

In [74]:
try:
    # Ensayar con una palabra que no está en el vocabulario:
    w2v_model.wv.most_similar(negative=["diedaa"])
    # Si no falla, lanzar una excepción manualmente (opcional)
    raise Exception("Este código debería haber fallado, pero no lo hizo.")
except KeyError as e:
    print(f"termino no encontrado, como se esperaba: -> {e}")


termino no encontrado, como se esperaba: -> "Key 'diedaa' not present in vocabulary"


In [75]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.17736822  0.21460655 -0.05408018  0.06707036  0.13059464 -0.25713933
  0.00711611  0.26743424 -0.03100746  0.259963    0.21179457 -0.08465387
 -0.26272097  0.308386   -0.07934437 -0.13357355  0.2994755   0.18062147
 -0.13111673 -0.11857472  0.03468326  0.1125607   0.08478875  0.08090881
 -0.17622267  0.03122034 -0.08725598  0.23746666 -0.11507358 -0.29119617
 -0.03650665 -0.3322312   0.23262788  0.10366543 -0.18365337  0.08299578
  0.14315338 -0.19761457 -0.00757894  0.1119568  -0.10421463 -0.13083619
 -0.03264419 -0.04687435  0.11330808  0.10231452 -0.16202034 -0.10347028
  0.03659901 -0.08524423 -0.34872082 -0.02154255  0.25938764  0.16540806
  0.0310861   0.09505633  0.24452917 -0.09859891  0.18298703  0.0645447
  0.016967   -0.3259168  -0.04774074 -0.05418535 -0.04338397 -0.09137302
  0.00889874  0.11636022 -0.25662988  0.05426472  0.08915784  0.04541444
  0.20835714 -0.19101828  0.32210803  0.19337991  0.09080223 -0.00362429
 -0.18664253 -0.07053486 -0.1213743  -0.03477469 -0.

In [76]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ('babe', 0.9132986068725586),
 ('someone', 0.8914992809295654),
 ('nothing', 0.8811879754066467),
 ('need', 0.8758756518363953),
 ("didn't", 0.8689683079719543),
 ("there's", 0.8557919859886169),
 ('you', 0.8447725772857666),
 ('feed', 0.8422104716300964),
 ('somebody', 0.8385539054870605)]

In [77]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('babe', 0.9132986068725586),
 ('someone', 0.8914992809295654),
 ('nothing', 0.8811879754066467),
 ('need', 0.8758756518363953),
 ("didn't", 0.8689682483673096),
 ("there's", 0.8557919859886169),
 ('you', 0.8447725772857666),
 ('feed', 0.8422104716300964),
 ('somebody', 0.8385538458824158),
 ('hope', 0.8361446261405945)]

### 5 - Visualizar agrupación de vectores

In [78]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [79]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

In [80]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() # esto para plotly en colab

In [81]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

He decidido emplear un dataset de tweets durante la época del coronavirus.

El dataset puede ser encontrado en el siguiente link: https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification
#### Importacion de librerías

In [9]:
from keras_preprocessing.text import text_to_word_sequence
from gensim.models.callbacks import CallbackAny2Vec
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np   
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from gensim.models import Word2Vec

Primero descomprimimos el fichero y leemos los datos

In [82]:
!unzip -q covid-tweet-dataset.zip

"unzip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [10]:
dataframe_tweets_train = pd.read_csv("Corona_NLP_train.csv", encoding='latin-1')
dataframe_tweets_test = pd.read_csv("Corona_NLP_test.csv", encoding='latin-1')
dataframe_tweets = pd.concat([dataframe_tweets_train, dataframe_tweets_test], ignore_index=True)
# Reemplazar '\r' por un string vacío en todas las columnas
dataframe_tweets = dataframe_tweets.replace('\r', '', regex=True)
dataframe_tweets.head(300)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
295,4094,49046,"Portland, Ore.",16-03-2020,Oregon Gov. Kate Brown on Monday afternoon ord...,Negative
296,4095,49047,Belgium,16-03-2020,#Coronavirus - #Europe calls for calm as food ...,Negative
297,4096,49048,Yorkshire,16-03-2020,When even Amazon is struggling to deliver food...,Extremely Negative
298,4097,49049,"Washington, D.C.",16-03-2020,Anthony Fauci said that while some people may ...,Negative


Obtenemos el texto de interés. En este caso, el contenido de los tweets.

In [11]:
dataframe_tweets = pd.DataFrame({'Text': dataframe_tweets["OriginalTweet"]})
dataframe_tweets.head(300)

,Text
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...
1,advice Talk to your neighbours family to excha...
2,Coronavirus Australia: Woolworths to give elde...
3,My food stock is not the only one which is emp...
4,"Me, ready to go at supermarket during the #COV..."
...,...
295,Oregon Gov. Kate Brown on Monday afternoon ord...
296,#Coronavirus - #Europe calls for calm as food ...
297,When even Amazon is struggling to deliver food...
298,Anthony Fauci said that while some people may ...


### 1 - Preprocesamiento

In [12]:
sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in dataframe_tweets[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

# Demos un vistazo
sentence_tokens[:2]

C:\Users\Lorda\AppData\Local\Temp\ipykernel_18148\4071598931.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence_tokens.append(text_to_word_sequence(row[0]))


[['menyrbie',
  'phil',
  'gahan',
  'chrisitv',
  'https',
  't',
  'co',
  'ifz9fan2pa',
  'and',
  'https',
  't',
  'co',
  'xx6ghgfzcc',
  'and',
  'https',
  't',
  'co',
  'i2nlzdxno8'],
 ['advice',
  'talk',
  'to',
  'your',
  'neighbours',
  'family',
  'to',
  'exchange',
  'phone',
  'numbers',
  'create',
  'contact',
  'list',
  'with',
  'phone',
  'numbers',
  'of',
  'neighbours',
  'schools',
  'employer',
  'chemist',
  'gp',
  'set',
  'up',
  'online',
  'shopping',
  'accounts',
  'if',
  'poss',
  'adequate',
  'supplies',
  'of',
  'regular',
  'meds',
  'but',
  'not',
  'over',
  'order']]

### 2 - Crear los vectores (word2vec)

In [13]:
# Se repite la definicion del callback en el caso de que solo se quieran ejecutar las celdas de resolucion del alumno
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [14]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de docs en el corpus: 44955
Cantidad de words distintas en el corpus: 12072


### 3 - Entrenar embeddings

In [15]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 9617147.0
Loss after epoch 1: 7747927.0
Loss after epoch 2: 7388836.0
Loss after epoch 3: 7236748.0
Loss after epoch 4: 7437562.0
Loss after epoch 5: 7358632.0
Loss after epoch 6: 7189584.0
Loss after epoch 7: 7049552.0
Loss after epoch 8: 6293212.0
Loss after epoch 9: 1651328.0
Loss after epoch 10: 1616440.0
Loss after epoch 11: 1572840.0
Loss after epoch 12: 1530560.0
Loss after epoch 13: 1490056.0
Loss after epoch 14: 1451904.0
Loss after epoch 15: 1407536.0
Loss after epoch 16: 1368184.0
Loss after epoch 17: 1326576.0
Loss after epoch 18: 1292016.0
Loss after epoch 19: 1269240.0


(21047777, 29517160)

### 4 - Ensayar

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["covid"], topn=10)

[('19', 0.6262304782867432),
 ('cyprus', 0.5430576205253601),
 ('cornavirusoutbreak', 0.5391762852668762),
 ('bedevils', 0.5248953700065613),
 ('pandemic2020', 0.5238670706748962),
 ('19ph', 0.5175456404685974),
 ('subside', 0.5174241662025452),
 ('staythefuckhome', 0.5157158374786377),
 ('ctv', 0.5146151781082153),
 ('covidbc', 0.5142259001731873)]

Podemos apreciar con claridad la cantidad de sinónimos, continuaciones o terminos que coinciden en contexto con el termino covid. Sospecho que varios de estos terminos irían acompañados de 'hashtag'(#), sobre todo aquellos que son varias palabras concatenadas. El único término extraño en la lista es 'bedevils', el cual solo puedo especular que tiene algo que ver con la rama negacionista de la pandemia.

In [17]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["healthy"], topn=10)

[('largest', 0.05206666886806488),
 ('accused', 0.041621968150138855),
 ('delay', 0.04105822369456291),
 ('washington', 0.03790722042322159),
 ('points', 0.03690763935446739),
 ('shift', 0.03120720572769642),
 ('petrol', 0.02896859496831894),
 ('inflate', 0.027529465034604073),
 ('usage', 0.025413915514945984),
 ('platform', 0.025325028225779533)]

Las palabras que menos se relacionan con saludable parecen aleatorias. Esto es lo esperable: No ser capaces de unir ambas palabras en una frase que tenga sentido. Tardé un poco en comprenderlo del todo porque esperaba que apareciera alguno de los antónimos de la palabra, pero esos términos guardan mucha más relacion con ella que cualquier otra que no tenga un contexto en común

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["hospital"], topn=10)

[("doctor's", 0.48108863830566406),
 ('nursery', 0.45239660143852234),
 ('emt', 0.45165005326271057),
 ('paramedics', 0.45159590244293213),
 ('emts', 0.44991549849510193),
 ('newyorkcity', 0.4455753266811371),
 ('er', 0.444820761680603),
 ('porters', 0.44025781750679016),
 ('bodega', 0.4343589246273041),
 ('techs', 0.4333254098892212)]

Hospital guarda relacion con nombres de ciudades, tambien con tecnicismos como emt (emergency medical technicians) y oficios relacionados con los servicios públicos (policias, conserjes, enfermeros, doctores...). El témino extraño es ER, el cual es una serie de televisión relacionada con un hospital https://en.wikipedia.org/wiki/ER_(TV_series)

In [19]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["paper"], topn=10)

[('toilet', 0.7060685753822327),
 ('towels', 0.5793225765228271),
 ('napkins', 0.5328953266143799),
 ('towel', 0.5128803253173828),
 ('tp', 0.50861656665802),
 ('elusive', 0.4916432499885559),
 ('roll', 0.48424217104911804),
 ('papertowels', 0.47417959570884705),
 ('charmin', 0.4730319380760193),
 ('bottled', 0.4727112948894501)]

De nuevo, terminos relacionados con el papel higiénico. Esperaba encontrar términos relacionados con el desabastecimiento durante la locura colectiva al comienzo de la pandemia, pero supongo que no tuvo suficiente impacto como para influenciar tanto al embedding.

In [20]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["pandemic"], topn=10)

[('simultaneous', 0.44074511528015137),
 ('deforestation', 0.44040003418922424),
 ('coronaapocalypse', 0.4383789598941803),
 ('outbreak', 0.4376335144042969),
 ('coronaviruscrisis', 0.43504559993743896),
 ('wreaks', 0.43199801445007324),
 ('foodinsecurity', 0.4319835603237152),
 ('nm', 0.4292699098587036),
 ('mapoli', 0.4292248785495758),
 ('fixturescloseup', 0.4199630916118622)]

Este término tiene una gran cantidad de hashtags y continuaciones esperables de la palabra (outbreak, crisis). Hay otros términos más difusos

In [21]:
try:
    # Ensayar con una palabra que no está en el vocabulario:
    w2v_model.wv.most_similar(negative=["halberd"])
    # Si no falla, lanzar una excepción manualmente (opcional)
    raise Exception("Este código debería haber fallado, pero no lo hizo.")
except KeyError as e:
    print(f"termino no encontrado, como se esperaba: -> {e}")

termino no encontrado, como se esperaba: -> "Key 'halberd' not present in vocabulary"


In [22]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("covid")
print(vector_love)

[-0.04078294  0.19822592  0.1632768  -0.02819409  0.09986134 -0.47223222
  0.34094355  0.38048753 -0.10183523 -0.19937275 -0.13838853 -0.0725431
 -0.0397372   0.00455932 -0.45309165 -0.23051317 -0.02019536 -0.01401536
  0.43203315  0.02644113 -0.17541759 -0.23279579  0.21667442  0.2664663
  0.27195948 -0.22395894 -0.22244625  0.07093975 -0.28635994  0.03699851
 -0.02996452 -0.10161694 -0.18854542  0.08377816 -0.28425294  0.01532153
 -0.43537718 -0.2347473  -0.1494438   0.16406463 -0.45976532 -0.12616242
 -0.3078208  -0.0476362   0.32304844 -0.0506929  -0.00558319  0.25035626
  0.2906951   0.05586935 -0.20313525 -0.05723209 -0.16456245 -0.15003099
 -0.1254802   0.17300421  0.1898833  -0.11201511  0.11730833  0.21636352
 -0.04960198 -0.16128355  0.111275   -0.01530833 -0.13047588  0.31417856
  0.06178514  0.21920103  0.16365881 -0.21051337 -0.12755269 -0.22786112
 -0.29998687  0.02848707  0.16013739 -0.02691453 -0.54313505 -0.43270123
  0.07014553  0.08740792  0.08967377 -0.09468219  0.4

### 5 - Visualizar agrupación de vectores

In [23]:
# Se vuelve a definir la funcion por si se quiere ejecutar solo las celdas de resolucion de alumno
def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [24]:
# Graficar los embedddings en 2D
vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

In [25]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() # esto para plotly en colab

### 6 - Análisis gráfico
Procedemos a analizar algunas de las imágenes que nos proporcionan las palabras vectorizadas en 2D y 3D:

![alt text](image-1.png)

En esta imagen podemos apreciar todas las palabras relacionadas con el covid guardando una cercanía notable. Curiosamente #toiletpaper esta muy lejana a las palabras toilet y paper. 

Es importante destacar que las gráficas son una simplificación de 200 puntos y que no cubren la totalidad del embedding utilizado. Quedan pendientes de realizar observaciones interesantes, como la representacion de los puntos que guardan menos relacion con uno dado, para apreciar las distancias entre ellos y otra algoritmia capaz de representar con mayor impacto clusters de puntos. Todo esto podría hacerse haciendo una selección de los puntos a graficar.

Otra nota: estas operaciones de reducción de dimensiones e impresion de graficas han supuesto un tiempo y esfuerzo considerables para la CPU de mi computadora.

### Conclusiones
Este notebook aporta unas algoritmia muy sencilla para el análisis exploratorio de un set de datos de texto con el uso de embeddings. La representacion de relaciones semanticas entre palabras, tanto de forma textual como a través de un 'mapa' vectorizado de las palabras, supone una herramienta muy poderosa a la hora de comprender la estructura de un corpus.